In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("CalculateItemScoreRanking") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/06/01 14:50:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 14:50:07 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [2]:
# define map functions 
from datetime import datetime, timedelta

today_string = datetime.today().strftime('%Y-%m-%d')
history_string = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
print(today_string)
print(history_string)

time_factor = 0.9

2023-06-01
2023-05-31


In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
event_score_data = [
    ('view', 1.0),
    ('click', 4.0),
    ('long_view', 7.0),
    ('add_to_favorites', 20.0),
    ('purchase', 30.0),
    ('search_view', 5.0),
    ('search_click', 10.0)
]

schema = StructType([
    StructField("event_type", StringType(), True),
    StructField("score", DoubleType(), True)
])

event_score_df = spark.createDataFrame(event_score_data, schema)
event_score_df.show()
event_score_df.createOrReplaceTempView("event_score")

+----------------+-----+
|      event_type|score|
+----------------+-----+
|            view|  1.0|
|           click|  4.0|
|       long_view|  7.0|
|add_to_favorites| 20.0|
|        purchase| 30.0|
|     search_view|  5.0|
|    search_click| 10.0|
+----------------+-----+



In [4]:
item_score_schema = StructType([
    StructField("item_id", StringType(), True),
    StructField("score", DoubleType(), True),
    StructField("date", StringType(), True)
])
spark.createDataFrame([],item_score_schema).createOrReplaceTempView("item_score")

In [5]:
item_score = spark.sql(f'''
with item_all as(
    select
        distinct id as item_id
    from
        item_ods
), history_item_score as(
    select
        a.item_id,
        if(b.item_id is null, 0, b.score) as score
    from
        item_all a
    left join
        item_score b on a.item_id = b.item_id
), today_item_score as(
    select
        a.item_id,
        sum(if(c.score is null, 0,c.score)) as score
    from
        item_all a
    left join
        event_ods b on a.item_id = b.item_id and b.timestamp = '{today_string}'
    left join
        event_score c on b.event_type = c.event_type
    group by
        a.item_id
    order by
        score desc
)
select
    a.item_id,
    b.score * {time_factor} + c.score as score,
    '{today_string}' as date
from
    item_all a
left join
    history_item_score b on a.item_id = b.item_id
left join
    today_item_score c on a.item_id = c.item_id
order by
    score desc
''')
item_score.show()

23/06/01 14:50:31 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+-------+-----+----------+
|item_id|score|      date|
+-------+-----+----------+
|     14|  0.0|2023-06-01|
|     18|  0.0|2023-06-01|
|     38|  0.0|2023-06-01|
|     46|  0.0|2023-06-01|
|     50|  0.0|2023-06-01|
|     73|  0.0|2023-06-01|
|     97|  0.0|2023-06-01|
|    161|  0.0|2023-06-01|
|    172|  0.0|2023-06-01|
|    186|  0.0|2023-06-01|
|    225|  0.0|2023-06-01|
|    232|  0.0|2023-06-01|
|    233|  0.0|2023-06-01|
|    248|  0.0|2023-06-01|
|    254|  0.0|2023-06-01|
|    257|  0.0|2023-06-01|
|    263|  0.0|2023-06-01|
|    280|  0.0|2023-06-01|
|    282|  0.0|2023-06-01|
|    300|  0.0|2023-06-01|
+-------+-----+----------+
only showing top 20 rows



In [6]:
item_score.createOrReplaceTempView("item_score")

In [7]:
item_order = spark.sql(f'''
with item_all as(
    select
        id as item_id,
        category
    from
        item_ods
)
select
    item_all.item_id,
    item_all.category,
    item_score.score,
    row_number() over(
        partition by
            category
        order by
            score desc
    ) as rn,
    '{today_string}' as date
from
    item_all
left join
    item_score on item_all.item_id = item_score.item_id
where
    date = '{today_string}'
order by
    rn
''')
item_order.show()
item_order.write.mode("overwrite").partitionBy("date").saveAsTable("item_score_rank")

+-------+---------------+-----+---+----------+
|item_id|       category|score| rn|      date|
+-------+---------------+-----+---+----------+
|    220|service_product|  0.0|  1|2023-06-01|
|     38|        service|  0.0|  1|2023-06-01|
|    164|       breeding|  0.0|  1|2023-06-01|
|     18| cattle_product|  0.0|  1|2023-06-01|
|     14|   whole_cattle|  0.0|  1|2023-06-01|
|    223|service_product|  0.0|  2|2023-06-01|
|    282|        service|  0.0|  2|2023-06-01|
|    363|   whole_cattle|  0.0|  2|2023-06-01|
|     46| cattle_product|  0.0|  2|2023-06-01|
|    165|       breeding|  0.0|  2|2023-06-01|
|    365|   whole_cattle|  0.0|  3|2023-06-01|
|    300|        service|  0.0|  3|2023-06-01|
|    173|       breeding|  0.0|  3|2023-06-01|
|    237|service_product|  0.0|  3|2023-06-01|
|     50| cattle_product|  0.0|  3|2023-06-01|
|    175|       breeding|  0.0|  4|2023-06-01|
|    243|service_product|  0.0|  4|2023-06-01|
|    307|        service|  0.0|  4|2023-06-01|
|     73| cat

In [8]:
spark.stop()